In [217]:
# importing required libraries
import pandas as pd
import numpy as np
import plotly

from pathlib import Path
# from pandas_profiling import ProfileReport

In [218]:
# Raw Data Directory
raw_product_dir = Path.cwd().parent / "data" / "raw"
raw_product_dir

PosixPath('/home/calkikhunt/Workspace/diamond_color_quality_prediction/data/raw')

In [219]:
# Wrong file format exception class
class WrongFileFormat(Exception):
    """Wrong file format exception class"""
    pass

In [220]:
# Create a data frame that consists of the whole dataset 
# and store it in a final CSV file
dfs = []
for file in raw_product_dir.glob("*"):
    if file.suffix == ".csv":
        temp_df = pd.read_csv(file, encoding="utf-8")
        dfs.append(temp_df)

    elif file.suffix == ".xlsx":
        temp_df = pd.read_excel(file)
        temp_df.columns = temp_df.columns.astype(str)
        dfs.append(temp_df)

    else:
        raise WrongFileFormat("Sorry, The wrong file format was provided ",
            "which is not being handled.")

df = pd.concat(dfs, ignore_index=True)

In [221]:
# Prepare final dataset file which combine entire dataset
# df.to_csv("./../data/raw/final_data.csv")

In [222]:
df.head()

,,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082,Unnamed: 1059
0,W_Calibration,100.0,0.0,0.0,95.0422,100.0,108.8760,565.016,99.9909,0.000017,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN
1,F_Calibration,100.0,0.0,0.0,108.1040,100.0,39.2965,491.070,99.9909,0.000170,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN
2,W_Calibration,100.0,0.0,0.0,95.0422,100.0,108.8760,565.016,99.9909,0.000017,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN
3,F_Calibration,100.0,0.0,0.0,108.1040,100.0,39.2965,491.070,99.9909,0.000170,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN
4,W_Calibration,100.0,0.0,0.0,95.0422,100.0,108.8760,565.016,99.9909,0.000017,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN


In [223]:
# Remove Unknown colume because it's does containe any data
df.drop(columns=["Unnamed: 1059"], inplace=True)

In [224]:
# Checking if any columns containe null value or not
# Still there is few null values we'll take care of those later
df.isnull().sum().sum()

472

In [225]:
# As per domain expert which row contain calibration 
# data is not usefull for prediction so we're removing that
# df.drop(index=np.where(df.iloc[:, 0].str.contains("Calibration"))[0], inplace=True)
df = df[~df.iloc[:, 0].str.contains("calibration|calibaration", case=False)]

In [226]:
# Check again if null value is still present 
# or not in dataframe
df.isnull().sum().sum()

472

In [227]:
# got to know null value is still present and its in 
# one singal row so removing that row
df.dropna(inplace=True)

In [228]:
# As per domain expert 1 to 15 columns is not necessary
# so that removing that column
df.drop(columns=df.columns[1:15], inplace=True)

In [229]:
df.head()

,,195.462,196.256,197.049,197.841,198.634,199.427,200.22,201.013,201.805,...,978.616,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082
10,W_TP12 | 299_D_None_0.501_SI1,-66.6667,115.263,43.0435,126.364,97.7980,97.8358,98.1102,98.1925,98.7381,...,100.010,99.4989,99.2816,99.3555,99.9267,99.6637,141.4290,-336.6670,162.72700,-87.1429
11,F_TP12 | 299_D_None_0.501_SI1,-300.0000,150.755,67.7852,259.517,100.5420,101.6540,102.1460,102.1690,101.9250,...,106.389,105.8270,104.3240,104.2880,108.2030,108.3840,54.2130,37.5465,-85.04030,-14.6165
12,W_3472A2 | 242_D_None_0.501_VS2,0.0000,139.825,11.1594,168.788,100.3780,100.5550,100.7750,100.5210,100.6260,...,101.946,102.0660,102.3670,102.4550,102.7540,102.2010,379.5240,618.8890,59.69700,-249.0480
13,F_3472A2 | 242_D_None_0.501_VS2,-300.0000,-275.529,-112.7520,268.278,97.2973,97.5773,98.1667,97.9190,97.9886,...,105.627,104.0840,104.0160,103.0710,105.2780,107.5160,68.6804,26.7658,8.28539,111.1430
14,W_3472A2 | 98_F_None_0.61_VS2,-66.6667,145.263,85.2174,323.636,96.0762,97.0676,96.9140,96.6270,96.2321,...,101.618,101.6590,101.6000,101.5270,101.8760,101.5030,-62.8571,253.3330,178.18200,-120.0000


In [237]:
split_name = df.iloc[:, 0].str.split("_").str
df["characteristics"], df["color"], df["quality"], df["size"] = split_name[0], split_name[2], split_name[3], split_name[4]

In [238]:
df

,,195.462,196.256,197.049,197.841,198.634,199.427,200.22,201.013,201.805,...,981.491,982.209,982.928,983.646,984.364,985.082,characteristics,color,quality,size
10,W_TP12 | 299_D_None_0.501_SI1,-66.6667,115.2630,43.04350,126.3640,97.7980,97.8358,98.1102,98.1925,98.7381,...,99.9267,99.6637,141.42900,-336.66700,162.72700,-87.1429,W,D,None,0.501
11,F_TP12 | 299_D_None_0.501_SI1,-300.0000,150.7550,67.78520,259.5170,100.5420,101.6540,102.1460,102.1690,101.9250,...,108.2030,108.3840,54.21300,37.54650,-85.04030,-14.6165,F,D,None,0.501
12,W_3472A2 | 242_D_None_0.501_VS2,0.0000,139.8250,11.15940,168.7880,100.3780,100.5550,100.7750,100.5210,100.6260,...,102.7540,102.2010,379.52400,618.88900,59.69700,-249.0480,W,D,None,0.501
13,F_3472A2 | 242_D_None_0.501_VS2,-300.0000,-275.5290,-112.75200,268.2780,97.2973,97.5773,98.1667,97.9190,97.9886,...,105.2780,107.5160,68.68040,26.76580,8.28539,111.1430,F,D,None,0.501
14,W_3472A2 | 98_F_None_0.61_VS2,-66.6667,145.2630,85.21740,323.6360,96.0762,97.0676,96.9140,96.6270,96.2321,...,101.8760,101.5030,-62.85710,253.33300,178.18200,-120.0000,W,F,None,0.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6901,F_1731A2 | 273_I_None_0.703_SI1,-0.0000,-2788.8900,-1.50892,226.1260,75.6860,76.1144,75.8240,75.2799,75.3177,...,79.3466,79.2135,127.10000,-2.98103,73.16820,159.6210,F,I,None,0.703
6904,W_3486A2 | 68 _K_Faint_0.710_VS2,200.0000,2440.0000,20.16530,110.2930,91.3520,91.6037,91.7435,91.6514,92.1698,...,101.4940,102.0290,152.34000,100.63200,196.73500,12.6425,W,K,Faint,0.710
6905,F_3486A2 | 68 _K_Faint_0.710_VS2,50.0000,51.3812,98.65770,103.2790,53.4341,53.6453,54.1035,54.0111,54.3129,...,56.5170,55.3278,-1.67598,300.00000,-106.78000,-62.6374,F,K,Faint,0.710
6906,W_TQ12 | 102 _K_None_0.901_SI1,700.0000,-7160.0000,20.16530,110.2930,83.6619,83.7839,83.5113,83.5980,83.7482,...,89.6920,90.1110,101.27700,24.84210,-195.10200,99.6891,W,K,None,0.901
